In [5]:
%load_ext autoreload
%autoreload 2

import argparse, os, sys
from datetime import datetime
import numpy as np
sys.path.append('../test')
from model import TestModel, evaluate
from oads_access.oads_access import OADS_Access, OADSImageDataset
sys.path.append('../..')
from RetinalResources.retina_model import RetinaCortexModel
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from torchvision import utils
from result_manager.result_manager import ResultManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
input_dir = '/home/niklas/projects/data/oads/mini_oads/'
output_dir = '/home/niklas/projects/oads_access/output/'
device = torch.device('cuda')
# device = torch.device('cpu')

In [7]:
result_manager = ResultManager(root=output_dir)

In [8]:
home = input_dir
oads = OADS_Access(home)

# get train, val, test split, using crops if specific size
size = (200, 200)
train_data, val_data, test_data = oads.get_train_val_test_split(use_crops=True, min_size=size, max_size=size)
input_channels = np.array(train_data[0][0]).shape[-1]

output_channels = len(oads.get_class_mapping())
class_index_mapping = {key: index for index, key in enumerate(list(oads.get_class_mapping().keys()))}


In [9]:
model_name = 'retina_cortex'
# model_name = 'test'

if model_name == 'test':
    # Initialize model
    model = TestModel(input_channels=input_channels, output_channels=output_channels, input_shape=size, kernel_size=(3,3))
elif model_name == 'retina_cortex':
    model = RetinaCortexModel(n_retina_layers=2, n_retina_in_channels=input_channels, n_retina_out_channels=2, retina_width=32,
                                input_shape=size, kernel_size=(9,9), n_vvs_layers=2, out_features=output_channels, vvs_width=32)

if device == torch.device('cuda'):
    model = torch.nn.DataParallel(model)
model = model.to(device)

In [95]:
def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
    n,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    fig = plt.figure( figsize=(nrow,rows) )
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    return fig

In [34]:
# # layer = 0
# for layer in range(4):
#     if type(model.module.layers[layer]) == nn.modules.Conv2d:
#         filter = model.module.layers[layer].weight.cpu().data.clone()
#     else:
#         continue
#     visTensor(filter, ch=0, allkernels=False)

#     plt.axis('off')
#     plt.title(f'Layer {layer}')
#     plt.ioff()
#     plt.show()

In [10]:
batch_size = 16

# Get the custom dataset and dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

traindataset = OADSImageDataset(data=train_data, class_index_mapping=class_index_mapping, transform=transform, device=device)
valdataset = OADSImageDataset(data=val_data, class_index_mapping=class_index_mapping, transform=transform, device=device)
testdataset = OADSImageDataset(data=test_data, class_index_mapping=class_index_mapping, transform=transform, device=device)

trainloader = DataLoader(traindataset, batch_size=batch_size, shuffle=True, num_workers=1)
valloader = DataLoader(valdataset, batch_size=batch_size, shuffle=False, num_workers=1)
testloader = DataLoader(testdataset, batch_size=batch_size, shuffle=False, num_workers=1)


criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, momentum=0.9)

In [51]:
result_manager.save_dataset_description(trainloader=trainloader, valloader=valloader, testloader=testloader)

AttributeError: 'torch.Size' object has no attribute 'tolist'

In [37]:
res = evaluate(testloader, model, criterion=criterion)

Got 18 / 128 with accuracy 14.06%


In [12]:
# result_manager.save_result(res, filename='result_dict.yml', overwrite=True)

In [13]:
# result_manager.save_model(model, filename='model', overwrite=True)

In [38]:
for epoch in range(int(15)):  # loop over the dataset multiple times
    print(f"Running epoch {epoch}")
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

            val_res = evaluate(valloader, model, criterion=criterion)
            print(f"Validation accuracy: {val_res['accuracy']}")

print(f'Finished Training with loss: {loss.item()}')

Running epoch 0
[1,    20] loss: 119.018
Got 9 / 128 with accuracy 7.03%
Validation accuracy: 0.0703125
[1,    40] loss: 0.027
Got 11 / 128 with accuracy 8.59%
Validation accuracy: 0.0859375
[1,    60] loss: 0.026
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
Running epoch 1
[2,    20] loss: 0.025
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
[2,    40] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
[2,    60] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
Running epoch 2
[3,    20] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
[3,    40] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
[3,    60] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
Running epoch 3
[4,    20] loss: 0.024
Got 37 / 128 with accuracy 28.91%
Validation accuracy: 0.2890625
[4,    40] loss: 0.024
Got 37 / 128 with accuracy 28.91%


KeyboardInterrupt: 

In [39]:
evaluate(testloader, model, criterion=criterion)

Got 25 / 128 with accuracy 19.53%


{'accuracy': 0.1953125,
 'batch_losses': {0: 2.329002618789673,
  1: 2.189633369445801,
  2: 2.0372684001922607,
  3: 2.0097250938415527,
  4: 2.1287879943847656,
  5: 2.8553290367126465,
  6: 2.0751585960388184,
  7: 2.3353168964385986}}

In [49]:
# # After training
# for layer in range(4):
#     if type(model.module.layers[layer]) == nn.modules.Conv2d:
#         filter = model.module.layers[layer].weight.cpu().data.clone()
#     else:
#         print(type(model.module.layers[layer]))
#         continue
#     visTensor(filter, ch=0, allkernels=True)

#     plt.axis('off')
#     plt.title(f'Layer {layer}')
#     plt.ioff()
#     plt.show()

Sequential(
  (0): GaussianNoise()
  (retina_0): Conv2d(3, 32, kernel_size=(9, 9), stride=(1, 1), padding=same)
  (retina_relu_0): ReLU()
  (retina_1): Conv2d(32, 2, kernel_size=(9, 9), stride=(1, 1), padding=same)
  (retina_relu_1): ReLU()
  (gaussian_noise_out): GaussianNoise()
)
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.nn.modules.linear.Linear'>


In [112]:
figs = []
for name, module in model.module.named_modules():
    if not isinstance(module, nn.Sequential):
        if type(module) == nn.modules.conv.Conv2d or type(module) == nn.Conv2d:
            filter = module.weight.cpu().data.clone()
        else:
            continue
        fig = visTensor(filter, ch=0, allkernels=True)
        figs.append(fig)
        plt.axis('off')
        plt.title(f'Layer: {name}')
        plt.ioff()
        # plt.show()

result_manager.save_pdf(figs=figs, filename='layer_visualisation_after_training.pdf')